In [25]:
import tensorflow as tf
import numpy as np

In [28]:
sess = tf.InteractiveSession()

In [29]:

seq_length = 5 # We will be encoding a sequence of 5 notes

embedding_dim = 50

batch_size = 102 # length of each tensor in the sequence (1x102)

memory_dim = 100

vocab_size = 2 # only allowed symbols are 0 or 1. 

In [30]:
enc_inp = [tf.placeholder(tf.int32, shape=(None,),
                          name="inp%i" % t)
           for t in range(seq_length)]

labels = [tf.placeholder(tf.int32, shape=(None,),
                        name="labels%i" % t)
          for t in range(seq_length)]

weights = [tf.ones_like(labels_t, dtype=tf.float32)
           for labels_t in labels]

# Decoder input: prepend some "GO" token and drop the final
# token of the encoder input
dec_inp = ([tf.zeros_like(enc_inp[0], dtype=np.int32, name="GO")]
           + enc_inp[:-1])

# Initial memory value for recurrence.
prev_mem = tf.zeros((batch_size, memory_dim))

In [31]:
cell = tf.nn.rnn_cell.LSTMCell(memory_dim,state_is_tuple = True)
outputs, memory = tf.nn.seq2seq.embedding_rnn_seq2seq(enc_inp, dec_inp, cell,vocab_size,vocab_size,embedding_dim)

ValueError: Variable embedding_rnn_seq2seq/RNN/EmbeddingWrapper/embedding already exists, disallowed. Did you mean to set reuse=True in VarScope? Originally defined at:

  File "<ipython-input-6-4d683a4fb711>", line 3, in <module>
    enc_inp, dec_inp, cell,vocab_size,vocab_size,embedding_dim)
  File "/Users/rchoudhury/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2885, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/Users/rchoudhury/anaconda/lib/python2.7/site-packages/IPython/core/interactiveshell.py", line 2825, in run_ast_nodes
    if self.run_code(code, result):


In [32]:
loss = tf.nn.seq2seq.sequence_loss(dec_outputs, labels, weights, vocab_size)

In [8]:
tf.scalar_summary("loss", loss)

<tf.Tensor 'ScalarSummary:0' shape=() dtype=string>

In [9]:
magnitude = tf.sqrt(tf.reduce_sum(tf.square(dec_memory[1])))
tf.scalar_summary("magnitude at t=1", magnitude)

<tf.Tensor 'ScalarSummary_1:0' shape=() dtype=string>

In [10]:
summary_op = tf.merge_all_summaries()

In [11]:
learning_rate = 0.05
momentum = 0.9
optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
train_op = optimizer.minimize(loss)

In [27]:
sess.close()